# Library

In [ ]:
# f12 -> console

# function ClickConnect(){
#     console.log("prevent colab disconnecting...");
#     document.querySelector("colab-toolbar-button#connect").click()
# }
# undefined
# setInterval(ClickConnect, 60*5000) // 5 minutes

In [ ]:
!pip install einops
!pip install timm

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as T
import pandas as pd
import numpy as np
from einops import rearrange # for changing channels

from glob import glob
import json
import csv
import cv2 # for image load
import albumentations as A
import albumentations.pytorch

import plotly.express as px # for graph

import timm # for pretrained models
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from sklearn.metrics import f1_score # for f1 score

from tqdm import tqdm
import time
from time import sleep

# WanDB

In [ ]:
!pip install wandb
import wandb
!wandb login

In [ ]:
# Get Test Dataset from Google Drive. (Approximately 3m)

!unzip "/content/drive/MyDrive/data" -d "/content/"
!unzip "/content/train" -d "/content/"
!unzip "/content/test" -d "/content/"

# ENV


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model_list = {
    "resnet50" : {
        "model" : "resnet50", 
        "input_size" : (224, 224),
        "classifier_in_feature" : 2048
        }, 
    "efficientnet_b0" : {
        "model" : "efficientnet_b0", 
        "input_size" : (224, 224),
        "classifier_in_feature" : 1280
        }, 
    "resnet18" : {
        "model" : "resnet18", 
        "input_size" : (224, 224),
        "classifier_in_feature" : 1024
        }, 
    "resnet50" : {
        "model" : "resnet50", 
        "input_size" : (224, 224),
        "classifier_in_feature" : 1024
        }, 
    "convnext_base_384_in22ft1k" : {
        "model" : "convnext_base_384_in22ft1k", 
        "input_size" : (384, 384),
        "classifier_in_feature" : 1024
        }
}

optimizer_list = ["adam", "sgd"]

model_name = "convnext_base_384_in22ft1k"

CONFIG = {
    "batch_size" : 16,
    "epoch" : 200,
    "learning_rate" : 1e-4,
    "input_size" : model_list[model_name]["input_size"],
    "backbone" : model_list[model_name]["model"],
    "classifier_in_feature": model_list[model_name]["classifier_in_feature"],
    "device" : device,
    "patience" : 10,
    "optimizer" : optimizer_list[0],
    "input_norm_mean" : IMAGENET_DEFAULT_MEAN,
    "input_norm_std" : IMAGENET_DEFAULT_STD
}

batch_size = CONFIG["batch_size"]
epochs = CONFIG["epoch"]
learning_rate = CONFIG["learning_rate"]
input_size = CONFIG["input_size"]
backbone = CONFIG["backbone"]
classifier_in_feature = CONFIG["classifier_in_feature"]
device = CONFIG["device"]
patience = CONFIG["patience"]
optimizer = CONFIG["optimizer"]
input_norm_mean = CONFIG["input_norm_mean"]
input_norm_std = CONFIG["input_norm_std"]

# Read the Data

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, json_dir_list, img_dir_list, feature_label, transform=None):
        self.json_dir_list = json_dir_list
        self.img_dir_list = img_dir_list
        self.transform = transform
        self.feature_label = feature_label # dictionary
        self.img_list = []
        
        for i in range(len(json_dir_list)):
            json_path, img_path = json_dir_list[i], img_dir_list[i]
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.transform:
                image = self.transform(image=image)['image']

            label = 0
            with open(json_path, "r") as j:
                i = json.load(j)
                feature = f"{i['annotations']['crop']}_{i['annotations']['disease']}_{i['annotations']['risk']}"
                label = self.feature_label[feature] # label : int

            self.img_list.append((image, label))

    def __len__(self):
        return len(self.img_dir_list)

    def __getitem__(self, idx):
        image, label = self.img_list[idx]

        return image, label #label type: int

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_dir_list, transform=None):
        self.img_dir_list = img_dir_list
        self.transform = transform

    def __len__(self):
        return len(self.img_dir_list)

    def __getitem__(self, index):
        img_path = self.img_dir_list[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

In [ ]:
# WANDB_TRAIN_DATA_PATH = train_artifact_dir
# WANDB_TEST_DATA_PATH = test_artifact_dir
COLAB_TRAIN_DATA_PATH = "/content"
COLAB_TEST_DATA_PATH = "/content/test"
# LOCAL_TRAIN_DATA_PATH = "./crop_data/train.csv"

TRAIN_DATA_PATH = COLAB_TRAIN_DATA_PATH
TEST_DATA_PATH = COLAB_TEST_DATA_PATH
Possible_comb_list = []
TRAIN_JSON_LIST, TRAIN_JPG_LIST = [], []

# Get Train Data

with open(TRAIN_DATA_PATH+"/train.csv", "r") as train_csv:
    i = csv.reader(train_csv, delimiter=',')
    for j in i:
        Possible_comb_list.append(j[1])

Possible_comb_list = list(set(Possible_comb_list[1:])) # length = 25
label_feature = {i: Possible_comb_list[i] for i in range(len(Possible_comb_list))} # dictionary
feature_label = {Possible_comb_list[i]: i for i in range(len(Possible_comb_list))} # dictionary

with open(TRAIN_DATA_PATH+"/train.csv", "r") as f:
    i = csv.reader(f, delimiter=',')
    for j in i:
        TRAIN_JSON_LIST.append(TRAIN_DATA_PATH+f"/train/{j[0]}/{j[0]}.json")
        TRAIN_JPG_LIST.append(TRAIN_DATA_PATH+f"/train/{j[0]}/{j[0]}.jpg")
TRAIN_JSON_LIST = TRAIN_JSON_LIST[1:]
TRAIN_JPG_LIST = TRAIN_JPG_LIST[1:]

# Get Test Data

TEST_JPG_LIST = []
with open(TRAIN_DATA_PATH+"/sample_submission.csv", "r") as f:
    i = csv.reader(f, delimiter=',')
    
    for j in i:
        TEST_JPG_LIST.append(TEST_DATA_PATH+f"/{j[0]}/{j[0]}.jpg")
TEST_JPG_LIST = TEST_JPG_LIST[1:]

In [ ]:
A_transform = A.Compose([
    A.Resize(input_size[0], input_size[1]),
    albumentations.augmentations.transforms.Normalize(
        mean=input_norm_mean, 
        std=input_norm_std
        ),
    albumentations.pytorch.transforms.ToTensorV2()
])

# numpy : h w c
# torch : c h w

labeled_dataset = CustomDataset(TRAIN_JSON_LIST, TRAIN_JPG_LIST, feature_label, transform=A_transform)
test_dataset = TestDataset(TEST_JPG_LIST, A_transform)

def dataset_split(labeled_dataset, ratio):
    train_size = int(ratio * len(labeled_dataset))
    test_size = len(labeled_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(labeled_dataset, [train_size, test_size])
    return train_dataset, val_dataset


# Early stopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path
        self.best_score = None

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}.")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Model

In [ ]:
class Model(nn.Module):
    def __init__(self, backbone, classifier_in_feature, pretrained=True):
        super(Model, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.classifier = nn.Linear(
                in_features=classifier_in_feature,
                out_features=25
            )
        
        for param in self.backbone.parameters():
            param.requires_grad=False

    def forward(self, images):
        output = self.backbone(images) # bs 1 1000
        output = self.classifier(output)
        return output


model = Model(backbone, classifier_in_feature, pretrained=True)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()

if optimizer == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
elif optimizer == "sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training & Validation Function

In [ ]:
def train_one_epoch(model, optimizer, loss_fn, dataloader, device):
    model.train()
    correct = 0
    running_loss = 0
    train_pred, train_y = [], []

    for (x, y) in enumerate(tqdm(dataloader)):
        x, y = x.to(device), y.to(device) # x shape: 8 3 224 224
        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss
        correct += torch.sum(pred.argmax(dim=1) == y)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()

    score = f1_score(train_y, train_pred, average='macro')

    acc = 100 * correct / (len(dataloader) * batch_size)
    running_loss = running_loss/len(dataloader)
    return running_loss, acc, score


In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device):
    model.eval()
    loss = 0
    running_loss = 0
    correct = 0
    val_pred, val_y = [], []

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y).item()

        running_loss += loss
        correct += torch.sum(pred.argmax(dim=1) == y)
        val_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        val_y += y.detach().cpu().numpy().tolist()

    score = f1_score(val_y, val_pred, average='macro')
        
    acc = 100 * correct / (len(dataloader) * batch_size)
    running_loss = running_loss/len(dataloader)

    return running_loss, acc, score

In [ ]:
@torch.no_grad()
def test(model, dataloader, device, result_csv):
    model.eval()
    rst = []
    for batch, x in enumerate(tqdm(dataloader)):
        x = x.to(device)
        pred = model(x)
        estimation = pred.argmax(dim=1)

        df = pd.read_csv(result_csv)
        for i in range(len(x)):
            rst.append((batch * len(x) + i, label_feature[estimation[i].item()]))
            df.loc[batch * len(x) + i, 'label'] = label_feature[estimation[i].item()]
        df.to_csv(result_csv, index=False)
    
    return rst


# Training

In [ ]:
run = wandb.init(
            project="crop_data",
            config=CONFIG
            ) # start a new run

In [ ]:
MODEL_PATH = "best_model.pt"
epoch_loss_list = []
train_loss_list = []
cur_f1_max = 0

early_stopping = EarlyStopping(patience=patience, verbose=False)

for e in range(epochs):
    print(f"\nEpoch {e+1}")
    train_dataset, val_dataset = dataset_split(labeled_dataset, 0.8) # 8 : 2 = train : val
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    val_dataloader = DataLoader(
        dataset=val_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    train_loss, train_acc, train_score = train_one_epoch(model, optimizer, loss_fn, train_dataloader, device)
    val_loss, val_acc, val_score = valid_one_epoch(model, val_dataloader, device)

    wandb.log({"train loss": train_loss, 
               "train accuracy": train_acc, 
               "validation loss": val_loss, 
               "validation accuracy": val_acc, 
               "train f1_score": train_score, 
               "validation f1_score": val_score})

    early_stopping(val_loss, model)
    print(f"train f1_score: {train_score}\tvalidation f1_score: {val_score}")

    if val_score > cur_f1_max:
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"Validation f1_score increased ({cur_f1_max:.6f} --> {val_score:.6f}). Saving model ...")
        cur_f1_max = val_score

    if early_stopping.early_stop:
        print("Early stopping")
        break

    epoch_loss_list.append(val_loss)
    train_loss_list.append(train_loss)


In [ ]:
# wandb.save(MODEL_PATH)

artifact = wandb.Artifact('model', type='model')
artifact.add_file(MODEL_PATH)
run.log_artifact(artifact)

wandb.finish() 

# Testing

In [ ]:
# # Get best Model

# run = wandb.init(project='crop_data')
# artifact = run.use_artifact('model:latest', type='model')
# artifact_dir = artifact.download()
# wandb.finish()
# MODEL_PATH = artifact_dir

model.load_state_dict(torch.load(MODEL_PATH))

In [ ]:
# Testing

print(f"Now Testing")
    
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

result = test(model, test_dataloader, device, TRAIN_DATA_PATH+"/sample_submission.csv")
print(result)

In [ ]:
# Uploading data to Wandb

upload = wandb.init(project="crop_data", name="uploading_result", config=CONFIG)

artifact = wandb.Artifact('crop_data_result', type='dataset')
artifact.add_file(TRAIN_DATA_PATH+"/sample_submission.csv")
upload.log_artifact(artifact)

wandb.finish()

# Train/Test Multiple time

In [ ]:
model_list = [
            #   ("resnet50", 2048), 
            #   ("efficientnet_b0", 1280), 
            #   ("resnet18", 512), 
            #   ("swin_base_patch4_window7_224_in22k", 1024),
            #   ("convnext_base_384_in22ft1k", 1024)
]

for i in range(len(model_list)):
    CONFIG["backbone"] = model_list[i][0]
    CONFIG["classifier_in_feature"] = model_list[i][1]
    backbone = CONFIG["backbone"]
    classifier_in_feature = CONFIG["classifier_in_feature"]

    model = Model(backbone, classifier_in_feature, pretrained=True)
    model = model.to(device)

    run = wandb.init(
            project="crop_data",
            config=CONFIG
            ) # start a new run

    MODEL_PATH = "best_model.pt"
    cur_f1_max = 0

    early_stopping = EarlyStopping(patience=patience, verbose=False)

    epoch_loss_list, train_loss_list = [], []

    for e in range(epochs):
        print(f"\nEpoch {e+1}")
        train_dataset, val_dataset = dataset_split(labeled_dataset, 0.8) # 8 : 2 = train : val
        train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=batch_size,
            shuffle=True
        )
        val_dataloader = DataLoader(
            dataset=val_dataset,
            batch_size=batch_size,
            shuffle=False
        )

        train_loss, train_acc, train_score = train_one_epoch(model, optimizer, loss_fn, train_dataloader, device)
        val_loss, val_acc, val_score = valid_one_epoch(model, val_dataloader, device)

        wandb.log({"train loss": train_loss, 
                "train accuracy": train_acc, 
                "validation loss": val_loss, 
                "validation accuracy": val_acc, 
                "train f1_score": train_score, 
                "validation f1_score": val_score})

        early_stopping(val_loss, model)
        print(f"train f1_score: {train_score}\tvalidation f1_score: {val_score}")

        if val_score > cur_f1_max:
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"Validation f1_score increased ({cur_f1_max:.6f} --> {val_score:.6f}). Saving model ...")
            cur_f1_max = val_score

        if early_stopping.early_stop:
            print("Early stopping")
            break

        epoch_loss_list.append(val_loss)
        train_loss_list.append(train_loss)
        # wandb.save(MODEL_PATH)

    artifact = wandb.Artifact('model', type='model')
    artifact.add_file(MODEL_PATH)
    run.log_artifact(artifact)

    # wandb.finish()

    # # Get best Model

    # run = wandb.init(project='crop_data')
    # artifact = run.use_artifact('model:latest', type='model')
    # artifact_dir = artifact.download()
    # wandb.finish()
    # MODEL_PATH = artifact_dir

    model.load_state_dict(torch.load(MODEL_PATH))

    print(f"Now Testing")

    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        shuffle=False
    )

    result = test(model, test_dataloader, device, TRAIN_DATA_PATH+"/sample_submission.csv")

    print(result)


    # Uploading data to Wandb

    # upload = wandb.init(project="crop_data")
    artifact = wandb.Artifact('crop_data_result', type='dataset')
    artifact.add_file(TRAIN_DATA_PATH+"/sample_submission.csv")
    upload.log_artifact(artifact)

    wandb.finish()
